## Documentation WongKinYiu/yolov7/tree/u7 and GitJvG/yolov7/tree/u7
Yolo v7 model
### Voorbereiding
Om het getrainde model te kunnen gebruiken, moet een compatibele python versie geïnstalleerd zijn en moet Git als PATH environment variable beschikbaar zijn.
- Python 3.12
- Git, geregistreerd als Windows PATH environment variable
Om yolov7 en benodigde libraries te installeren kan setup/setup.bat gebruikt worden in de doelbestemming. Plaats het .bat bestand in de doelbestemming en dubbelklik het script. Kies vervolgens voor standard Yolov7 of een aangepaste Yolov7. De aangepaste repository heeft enkele functies gesplitst in meerdere functies om het makkelijker te integreren met een UI en om onnodige acties te voorkomen.
Het clonen van de benodigde repositories, het aanmaken van een python virtual environment en het installeren van de benodigde modules kan ook worden gedaan met behulp van de setup/setup_yolov7.bat, beschikbaar op de github repository.

## Trainen
De training module van Yolo v7 is geschreven als een command line tool, vandaar is het makkelijker om deze te gebruiken vanaf de commandline of middels jupyter magic commands.

Om nieuwe weights te trainen kan de volgende code gebruikt worden, de belangrijkste parameters worden toegelicht:

- epochs: Het aantal keren dat het model de dataset processed, hoe hoger hoe beter het model zich kan afstemmen op de data. Te veel epochs kan leiden tot overfitting.
- data: Het pad naar de data.yaml, e.g. `root/dataset/data.yaml`
- weights: Het pad naar de weights, e.g. `root/yolov7/runs/train-seg/custom/weights/best.pt`
- device: De hardware dat het model gebruikt. 0 voor GPU 1, cpu om de cpu van het apparaat te gebruiken. De GPU moet een nvidia GPU met cuda-cores zijn.
- project: De directory waarin de resultaten worden opgeslagen, e.g. `C:\yolov7`
- name: De naam waaronder de resultaten worden opgeslagen, e.g. `custom` -> `{project}/custom/{filename}`
- exist-ok: Wanneer deze parameter wordt gebruikt wordt de name folder opniew gebruikt als deze al bestaat, anders wordt geitereert: `{project}/custom2/{filename}`
- patience: Activeert earlystopping, het bijgevoegde getal is het aantal epochs zonder verbetering voordat de training automatisch stopt. 


In [ ]:
"""Training met jupyter magic commands"""

%cd yolov7

!python yolov7/seg/segment/train.py --batch 1 \
--epochs 300 \
--data {DATASET} \
--weights {WEIGHTS_PATH} \
--device 0 \
--project custom \
--name custom \
--exist-ok
--patience 30



"""commandline equivalent:
cd yolov7
python yolov7/seg/segment/train.py --data "C:..\dataset\data.yaml" --weights "C:..\weights.pt" --device 0 --name custom --project custom --exist-ok --patience 30

"""

## Inference
Om het model afbeelding te laten annoteren kan de volgende code gebruikt worden met de volgende parameters:

- weights: Het pad naar de weights, e.g. `root/yolov7/runs/train-seg/custom/weights/best.pt`
- conf: minimum confidence threshold. Een lagere threshold leidt tot meer detecties en een lagere accuraatheid.
- source: Het pad naar een afbeelding of `0` om de webcam als source te gebruiken

In [ ]:
%cd {ROOT}/yolov7
!python segment/predict.py \
--weights {ROOT}/yolov7/weights/best.pt \
--data dataset/data.yaml
--conf-thres 0.35 \
--iou_thres 0.35 \
--source {Dataset}/test/images \
--project OutputDirectory \
--name OutputFolder \
--exist-ok \

%cd {ROOT}

## Aangepaste modules t.o.v. originele repository

Om inference flexibeler te maken is een custom-predict module gemaakt, hierin zijn het laden van het model en de inference gesplitst. Hierdoor is het mogelijk om resultaten in realtime weer te geven en is het voor een website of app mogelijk om het model éém keer te initializeren tijdens opstart en vervolgens te hergebruiken.

Het gebruik van de aangepaste module werkt grotendeeels hetzelfde als de normale module behalve dat het model en device met behulp van een nieuwe functie worden geladen.
Daarnaast zijn er hooks toegevoegd waardoor de GUI de processing voortgang kan tracken en kunnen resultaten optioneel gesplitst worden op detcties/geen detecties.

In [ ]:
from yolov7.seg.segment.modular_predict import run, load_model

model, device = load_model(
    weights=r"yolov7\weights\best.pt",
    data = r"dataset\data.yaml",
    device='',
    update=False
)

run(
    model = model, # Model van de bovenstaande functie
    device = device, # Device van de bovenstaande functie
    source="/source", # Afbeelding, video, folder of `0` voor webcam/streaming
    conf_thres=0.35, # Minimum confidence voor detectie
    iou_thres=0.35, # Threshold voor het combineren van bounding boxes, een hogere threshold zorgt voor meer clustering en minder apparate bounding boxes
    nosave=False, # Output wel/niet opslaan
    project="", # Directory waarin de 'name' folder komt
    name='dir', # Directory waarin de inference resultaten komen. Het volledige pad is {project}/{name}/file.png
    exist_ok=True, # True: gebruikt bestaande name folder als deze bestaat, False: itereert wanneer {name} al bestaat, ouput wordt dan: {project}/{name}2/file.png
    )